# Similarity

In [206]:
# Librairies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import random
import time

from Preprocessing import *

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split, cross_val_score

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 60)

## Load data

In [207]:
# Data labellisées (catégories Innoscape)
df = pd.read_csv('../Data/t3_all_fr_prices_posweekly_prod_without_price.csv')

# Colonnes à garder
col = ['t3_all_fr_prices_posweekly_prod.productfamily_seller',
       't3_all_fr_prices_posweekly_prod.prodcategory4_seller',
       't3_all_fr_prices_posweekly_prod.prodcategory3_seller',
       't3_all_fr_prices_posweekly_prod.prodcategory2_seller',
       't3_all_fr_prices_posweekly_prod.prodcategory1_seller',
       't3_all_fr_prices_posweekly_prod.productseller_name',
       't3_all_fr_prices_posweekly_prod.id_source',
       't3_all_fr_prices_posweekly_prod.brand_seller']

df = df[col]

# Raccourcir nom des colonnes
raccourci_col = map(lambda string: string.replace('t3_all_fr_prices_posweekly_prod.',''), col)
col_name_dict = dict(zip(col, raccourci_col))

# Renommer nom des colonnes
df.rename(columns=col_name_dict, inplace=True)

df = df.fillna("")

## Tests/Exemples

In [208]:
# Exemples interessants : plusieurs mesures
print('des :', df['productseller_name'].iloc[26])
print('des :', df['productseller_name'].iloc[78])
print('des :', df['productseller_name'].iloc[30843])


des : Ryobi-RXPR01 Podadora-chaîne 254 mm-larg. 150 cm DIAMETRE-Coupe 20 cm
des : Nortene 172587 0,8 mm x 50 m Fil Plastifié Support utilisé pour filets – Vert
des : MASTIC COLLE POUR MUR ET FACADE SIKAFLEX 310ML TON PIERRE


In [209]:
# TEST
test = 'colle maxtack tuile 300 ml'
x = re.search("([\d.]+)\s*(\bm\b|ml)", test)

if x is not None:
    print(x)
else:
    print('Non')

<_sre.SRE_Match object; span=(20, 26), match='300 ml'>


## Preprocessing

In [211]:
# Lettre en minuscule
def str_lower(df):
    for col in df.columns:
        df[col] = df[col].str.lower()
    return df

df = str_lower(df)

In [212]:
# Récupérer les attributs : Poids, Mesure, Volume, Couleur
def preprocessing_attributs(df, name_col):
    df['Poids'] = df[name_col].apply(lambda x: re.search("([\d.,]+)\s*(g|kg)", x).group() if re.search("([\d.,]+)\s*(g|kg)", x) is not None else '')
    df['Mesure'] = df[name_col].apply(lambda x: re.search("([\d.,]+)\s*(mm|cm|\bm\b)", x).group() if re.search("([\d.,]+)\s*(mm|cm|\bm\b)", x) is not None else '')
    df['Volume'] = df[name_col].apply(lambda x: re.search("([\d.,]+)\s*(ml|cl|l)", x).group() if re.search("([\d.,]+)\s*(ml|cl|l)", x) is not None else '')
    df['Couleur'] = df[name_col].apply(lambda x: re.search("(gris|noir|rouge|blanc|bleu|vert|argent|multicolore)", x).group() if re.search("(gris|noir|rouge|blanc|bleu|vert|argent|multicolore)", x) is not None else '')
    return df

df = preprocessing_attributs(df, 'productseller_name')


In [213]:
# Normaliser unité de mesure 
# Exemple : poids en gramme, mesure en mètre, volume en litre
def normalization_unit(df, name_col):
    
    dico = {'Poids':{'g':1, 'kg':1000}, 'Mesure':{'mm':0.001, 'm':1, 'cm':0.01}, 'Volume':{'ml':0.001, 'cl':0.01, 'l':1}}
    dico_regex = {'Poids':'(g|kg)', 'Mesure':'(mm|cm|m)', 'Volume':'(ml|cl|l)'}
    
    df[name_col + '_unit'] = df[name_col].apply(lambda x: float(re.search('\d+', x).group())*dico[name_col][re.search(dico_regex[name_col], x).group()] if re.search("\d+", x) and re.search(dico_regex[name_col], x) is not None else '')
    
    return df

df = normalization_unit(df, 'Poids')
df = normalization_unit(df, 'Mesure')
df = normalization_unit(df, 'Volume')

In [216]:
# Ajouter les marques 
def brand(df):
    
    # Liste des marques
    dico = '/Users/valentinphetchanpheng/Documents/MS BGD Telecom ParisTech/Projet Fil Rouge/INNOSCAPE-master /Data/brand_new_names.txt'
    brands = open(dico).read().split()
    brands = [x.lower() for x in brands]
    unique_brand = list(set(brands))
    unique_brand.remove('""')
    regex = '(' + '|'.join(unique_brand) + ')'
    
    # Marque pour chaque produit
    df['brand'] = df['productseller_name'].apply(lambda x: re.search(regex, x).group() if re.search(regex, x) is not None else '')

    index_no_brand = df[df.brand == ''].index.tolist()
    for index in index_no_brand:
        df['brand'].iloc[index] = re.search(regex, df['brand_seller'].iloc[index]).group() if re.search(regex, df['brand_seller'].iloc[index]) is not None else ''
        
    return df


In [217]:
df = brand(df)

In [219]:
df = df.drop(['Poids', 'Mesure', 'Volume', 'Couleur'],1)

In [220]:
df.head(30)

productfamily_seller  \
0                      colles à carrelage   
1                                           
2                                           
3                                  mastic   
4                                   cires   
5                                           
6                       perceuse-visseuse   
7                   produits d'étanchéité   
8                               arroseurs   
9                                           
10                                          
11                  pompes pour fontaines   
12                   sécateurs et ciseaux   
13    accessoires pour outillage sans fil   
14  pièces et accessoires pour chaudières   
15                                          
16     filtres pour aspirateurs portatifs   
17                      meuleuses d'angle   
18                                          
19                               conduits   
20                               conduits   
21                        mastic au latex   
22                     pièces et raccords   
23                   casques et écouteurs   
24                   bordures pour jardin   
25                    marteau perforateur   
26                          tronçonneuses   
27                        stickers muraux   
28                     colles à carrelage   
29                                          

                     prodcategory4_seller  \
0                      colles à carrelage   
1                                           
2                                           
3                                  mastic   
4                                   cires   
5                                           
6                               perceuses   
7                   produits d'étanchéité   
8                               arroseurs   
9                                           
10                                          
11                                 pompes   
12                   sécateurs et ciseaux   
13    accessoires pour outillage sans fil   
14  pièces et accessoires pour chaudières   
15                                          
16                                filtres   
17                              meuleuses   
18                                          
19                               conduits   
20                               conduits   
21                                 mastic   
22                  tuyaux et accessoires   
23                   casques et écouteurs   
24                   bordures pour jardin   
25                              perceuses   
26                          tronçonneuses   
27                    décorations murales   
28                     colles à carrelage   
29                                          

                          prodcategory3_seller  \
0                           colles et adhésifs   
1                                                
2             accessoires pour lave-vaisselles   
3                           colles et adhésifs   
4             peintures, teintures et solvants   
5                                                
6                    outillage électroportatif   
7                           colles et adhésifs   
8                          matériel d'arrosage   
9                                                
10                            outillage à main   
11                          bassins d'agrément   
12                         outillage de jardin   
13  accessoires pour outillage électroportatif   
14                  chauffage et climatisation   
15                           tables de cuisson   
16                accessoires pour aspirateurs   
17                   outillage électroportatif   
18                    matériel de construction   
19                  chauffage et climatisation   
20                  chauffage et climatisation   
21                          colles et adhésifs   
22                         matériel d'arrosage   
23                           

In [21]:
# Test
dico = {'Poids':{'g':1, 'kg':1000}, 'Mesure':{'mm':0.001, 'm':1, 'cm':0.01}, 'Volume':{'ml':0.001, 'cl':0.01, 'l':1}}

test = '25g'
if re.search('\d+', test) and re.search('(g|kg)', test) is not None:
    print(float(re.search('\d+', test).group())*dico['Poids'][re.search('(g|kg)', test).group()])
else:
    print('non')


25.0
